In [1]:
pip install ipykernel


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Minimal cell to verify and reload spotify_api.py in Jupyter

import os
import importlib
import spotify_api
from datetime import datetime

# 1. Show which file Python is importing
module_path = getattr(spotify_api, '__file__', None)
print("Python is importing:", module_path)

# 2. Show the file's modification time
if module_path and os.path.exists(module_path):
    mtime = os.path.getmtime(module_path)
    print("File modification time (UTC):", datetime.utcfromtimestamp(mtime))
    print("File modification time (local):", datetime.fromtimestamp(mtime))

# 3. Reload the module to pick up any changes
importlib.reload(spotify_api)
from spotify_api import SpotifyAPI, SpotifyAPIProxy

print("Module reloaded successfully. You can now instantiate your classes.")


ModuleNotFoundError: No module named 'spotify_api'

In [7]:
token = "BQCP7KTcCJ4aLw_XHpwgzp5fmVIZxZlU4HcEJtWOnqzoTMvJlYwYQD4VAQ8SvgJR4jsA67k6wjyHLTCwew0dH967lIELC0wDq7PvgPblELVJ3oaxmfDi9mz0HrBOPyD_ZxoBP5t5R1g"

In [4]:
import os
print(os.getcwd())
%cd backend/


/workspaces/TrackRecord_FM
/workspaces/TrackRecord_FM/backend


In [5]:
import importlib
import spotify_api
importlib.reload(spotify_api)
%reload_ext autoreload
%autoreload 2
from spotify_api import SpotifyAPI, SpotifyAPIProxy

In [ ]:
!ls -l spotify_api.py


-rw-rw-rw- 1 codespace root 3809 Nov  6 23:34 spotify_api.py


In [ ]:
!pwd


/workspaces/TrackRecord_FM/backend


In [ ]:
!stat /workspaces/TrackRecord_FM/backend/spotify_api.py

  File: /workspaces/TrackRecord_FM/backend/spotify_api.py
  Size: 3809      	Blocks: 8          IO Block: 4096   regular file
Device: 7,4	Inode: 1835073     Links: 1
Access: (0666/-rw-rw-rw-)  Uid: ( 1000/codespace)   Gid: (    0/    root)
Access: 2025-11-06 23:34:50.425601496 +0000
Modify: 2025-11-06 23:34:30.024602286 +0000
Change: 2025-11-06 23:34:47.133601624 +0000
 Birth: 2025-11-06 23:34:30.024602286 +0000


In [8]:
api = SpotifyAPI(access_token=token)
print(api.get_token())
# proxy = SpotifyAPIProxy(api)

NameError: name 'access_token' is not defined

In [ ]:
# show long listing with full timestamp (more precise)
!ls -l --full-time spotify_api.py

# show inode, size, and all time fields
!stat spotify_api.py

# show first 200 lines (verify content and constructor signature)
!sed -n '1,200p' spotify_api.py


-rw-rw-rw- 1 codespace root 3893 2025-11-07 00:08:41.181851470 +0000 spotify_api.py


  File: spotify_api.py
  Size: 3893      	Blocks: 8          IO Block: 4096   regular file
Device: 7,4	Inode: 1835073     Links: 1
Access: (0666/-rw-rw-rw-)  Uid: ( 1000/codespace)   Gid: (    0/    root)
Access: 2025-11-07 00:09:03.086851244 +0000
Modify: 2025-11-07 00:08:41.181851470 +0000
Change: 2025-11-07 00:08:58.622851290 +0000
 Birth: 2025-11-07 00:08:41.181851470 +0000
import requests
from abc import ABC, abstractmethod
import time
from typing import Dict, Any, Optional

class APIInterface(ABC):
    @abstractmethod
    def fetch_api(self, *args, **kwargs):
        pass

class SpotifyAPIProxy(APIInterface):
    def __init__(self, api: APIInterface):
        self.api = api
        self.cache = {}
        self.base_url = "https://api.spotify.com/v1"
        self.access_token = self.api.get_token()
        pass
    
    def fetch_api(self, endpoint, access_token=None, method="GET", data=None, params=None) -> Dict[str, Any]:
        """
        Calls the Spotify Web API with OAuth 